In [1]:
import torch

In [2]:
# # Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,2,3,4,], dtype=torch.long)

# # y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1., 1., 1., 1., 1.,],
 [1., 1., 1., 1., 1.,],
 [1., 1., 1., 1., 1.,],
 [1., 1., 1., 1., 1.,],
 [1., 1., 1., 1., 1.,],
 [1,  1., 1., 1., 1.,],
 [1., 1., 1., 1., 1.,],
 [1., 1., 1., 1., 1.,],],
 dtype=torch.float32)

In [3]:
# создание последовательности, включающей blank символ
def create_lexicon(l):
    lexicon = torch.zeros(2*l.size()[0]+1)  
    j = 0
    for i in range(l.size()[0] * 2 + 1):
        if(i % 2 == 1):
            lexicon[ i ] = l[ j ]
            j+=1
    return lexicon

In [4]:
# инициализация первого столбца предсказанными вероятностями
def initialize_first_column(res, y):
    res[ 0 ][ 0 ] = y[ 0 ][ y.size(1) - 1 ]
    res[ 1 ][ 0 ] = y[ 0 ][ 0 ]
    return res

In [5]:
# рассчет дистанции от текущего символа до конца последовательности
def distance(l, s):
    d = 0
    while s < l.size(0) - 1:
        if l[ s ] == 0: # встречаем blank -> идем дальше
            s += 1
        elif s + 2 >= l.size(0): # дошли до границы, возвращаем высчитанную дистанцию 
            return d
        elif l[ s ] == l[ s + 2 ]: # если текущий символ равен символу через один символ, перемещаемся на один символ
            s += 1
        else: # если текущий символ не равен символу через один символ, перемещаемся на два символа
            s += 2
        d += 1
        
    return d

In [6]:
# проверка валидности пути из данной точки таблицы
def is_valid(l, T, s, t): 
    if (s < l.size(0) - 2 * (T- t) - 1) or distance(l, s) > (T - t - 1):
        return True
    return False   

In [7]:
# значение вероятности в ячейке
def get_alpha(res,t,s):
    if t >= 0 and s >= 0: 
        return res[ s ][ t ].item()
    else: # если вышли за границы тензора
        return 0

In [8]:
def compute_ctc_alpha(l, y):

    res = initialize_first_column(torch.zeros((2*l.size()[0]+1, y.size()[0])), y) # матрица итоговых вероятностей
    lexicon = create_lexicon(l) # последовательность, включающая blank символ

    for s in range(res.size(0)):
        for t in range(1, res.size(1)):    
            if is_valid(lexicon, y.size(0), s, t):
                res[s, t] = 0
            elif ((lexicon[s] == 0 or lexicon[s-2] == lexicon[s])):
                res[s, t] = (res[s, t-1] + res[s-1, t-1]) * y[t, int(lexicon[s].item() - 1 + y.size(1)) % y.size(1)]
            else:
                res[s, t] = (res[s, t-1] + res[s-1, t-1] + res[s-2, t-1]) * y[t, int(lexicon[s].item() - 1 + y.size(1) ) % y.size(1)]
        
    return res

In [9]:
# Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha
def test():
    al = compute_ctc_alpha(l, y)
    ritght_al = torch.tensor([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
                              [ 1.,  2.,  3.,  0.,  0.,  0.,  0.,  0.],
                              [ 0.,  1.,  3.,  0.,  0.,  0.,  0.,  0.],
                              [ 0.,  1.,  4., 10.,  0.,  0.,  0.,  0.],
                              [ 0.,  0.,  1.,  5., 15.,  0.,  0.,  0.],
                              [ 0.,  0.,  0.,  1.,  6., 21.,  0.,  0.],
                              [ 0.,  0.,  0.,  0.,  1.,  7.,  0.,  0.],
                              [ 0.,  0.,  0.,  0.,  1.,  8., 36.,  0.],
                              [ 0.,  0.,  0.,  0.,  0.,  1.,  9.,  0.],
                              [ 0.,  0.,  0.,  0.,  0.,  1., 10., 55.],
                              [ 0.,  0.,  0.,  0.,  0.,  0.,  1., 11.]])
    
    if torch.all(al == ritght_al):
        return True
    else:
        return False
    
assert test(), "Test failed"
    